<a href="https://colab.research.google.com/github/jvataidee/PesquisaOperacional/blob/master/agregacao_sintese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**by: [João Ataíde](https://www.joaoataide.com)**
# **Métodos de Agregação e Síntese**

---



In [1]:
#importar bibliotecas
import pandas as pd
import numpy as np
import os
from itertools import combinations, product

## **Método de Borda**


In [2]:
criterios = ["Manutencao", "Customizacao", "Aderência"]

alt1 = [ "4°" , "2°", "3°"]
alt2 = [ "2°" , "4°", "1°"]
alt3 = [ "1°" , "1°", "2°"]
alt4 = [ "3°" , "3°", "4°"]

In [3]:
matriz_decisao = pd.DataFrame([alt1, alt2, alt3, alt4], columns = criterios)

In [4]:
matriz_decisao

,Manutencao,Customizacao,Aderência
0,4°,2°,3°
1,2°,4°,1°
2,1°,1°,2°
3,3°,3°,4°


### **Transformar Ordenal em Número**


In [5]:

matriz_decisao_num = matriz_decisao.copy()

for coluna in criterios:
    for linha in range(len(matriz_decisao[coluna])):
        matriz_decisao_num[coluna][linha] = int(matriz_decisao_num[coluna][linha][0:1])

matriz_decisao_num

,Manutencao,Customizacao,Aderência
0,4,2,3
1,2,4,1
2,1,1,2
3,3,3,4


### **Coluna de soma e resultado**

In [6]:
matriz_decisao_num["Soma"] = matriz_decisao_num["Manutencao"] + matriz_decisao_num["Customizacao"] + matriz_decisao_num["Aderência"] 

In [7]:
resultado = matriz_decisao_num.sort_values("Soma").copy()
resultado["Resultado"] = ''

for i in range(len(resultado["Soma"])):
    resultado["Resultado"][i] = f'{i+1}°'

resultado

,Manutencao,Customizacao,Aderência,Soma,Resultado
2,1,1,2,4,3°
1,2,4,1,7,2°
0,4,2,3,9,1°
3,3,3,4,10,4°


## **Método Condorcet**


In [152]:
criterios = ["Alternativa","Infra", "Serviço", "Acesso"]

#Em Escala de Likert
''' 5 Muito bom, 4 Bom, 3 Neutro, 2 Ruim, 1 Muito Ruim '''
alt1 = ["Alternativa 1", 1, 4, 3]
alt2 = ["Alternativa 2",1, 4, 5]
alt3 = ["Alternativa 3",5, 5, 4]
alt4 = ["Alternativa 4",3, 5, 2]

matriz_decisao = pd.DataFrame([alt1, alt2, alt3, alt4], columns = criterios)
matriz_decisao

,Alternativa,Infra,Serviço,Acesso
0,Alternativa 1,1,4,3
1,Alternativa 2,1,4,5
2,Alternativa 3,5,5,4
3,Alternativa 4,3,5,2


In [153]:
alternativas = list(matriz_decisao.index)
criterios = list(matriz_decisao.columns[1:])
alternativas_combinadas =  list(combinations(alternativas, 2))
lista_alter = list(matriz_decisao.Alternativa)

In [154]:
dataframes = {
    criterio: pd.DataFrame(data = 0, index = alternativas, columns = alternativas)
    for criterio in criterios
}

In [155]:
for criterio, (altA, altB) in product(criterios, alternativas_combinadas):

    if matriz_decisao.at[altA, criterio] > matriz_decisao.at[altB, criterio]:
        dataframes[criterio].at[altA, altB] = 1

    elif matriz_decisao.at[altA, criterio] < matriz_decisao.at[altB, criterio]:
        dataframes[criterio].at[altA, altB] = -1

    else :
        dataframes[criterio].at[altA, altB] = 0

In [156]:
for criterio in criterios:
    dataframes[criterio].reset_index()
    dataframes[criterio].columns =  lista_alter
    dataframes[criterio].index = lista_alter

In [157]:
for criteiro in criterios:
    print(" ")
    print(criteiro)
    print(dataframes[criteiro])

 
Infra
               Alternativa 1  Alternativa 2  Alternativa 3  Alternativa 4
Alternativa 1              0              0             -1             -1
Alternativa 2              0              0             -1             -1
Alternativa 3              0              0              0              1
Alternativa 4              0              0              0              0
 
Serviço
               Alternativa 1  Alternativa 2  Alternativa 3  Alternativa 4
Alternativa 1              0              0             -1             -1
Alternativa 2              0              0             -1             -1
Alternativa 3              0              0              0              0
Alternativa 4              0              0              0              0
 
Acesso
               Alternativa 1  Alternativa 2  Alternativa 3  Alternativa 4
Alternativa 1              0             -1             -1              1
Alternativa 2              0              0              1              1
Alternativa

In [162]:
resultado = dataframes['Infra'] + dataframes['Serviço'] + dataframes['Acesso']
resultado = resultado['Alternativa 1'] + resultado['Alternativa 2'] + resultado['Alternativa 3'] + resultado['Alternativa 4']

for i in range(len(resultado)):
    if resultado[i] == 0:
        resultado[i] = 0

    elif resultado[i] > 0:
        resultado[i] = 1
        
    else:
        resultado[i] = -1

resultado

Alternativa 1   -1
Alternativa 2   -1
Alternativa 3    1
Alternativa 4    0
dtype: int64

## **Método Copeland**
